# Merging of Videogame review and sales info
### This is the notebook focused on merging the number of data sets.

Loading all the libraries. Keeping them up here and will update as I add them.

In [164]:
import csv
import numpy as np
import math as ma
import pandas as pd
import fuzzymatcher

### Loading all the data sets. 
This will be where I load all of them, so I can keep them together.

In [426]:
vg1 = pd.read_csv(r'Data/vgchartz-7_7_2020.csv')

vg2 = pd.read_csv(
    r'Data/Video_Games_Sales_as_at_22_Dec_2016.csv')

# # REMOVING VG3, because repeats VG1
# vg3 = pd.read_csv(
#     r'Data/vgchartz-6_23_2020.csv')

vg4 = pd.read_csv(
    r'Data/metacritic_games.csv')

vg5 = pd.read_csv(
    r'Data/metacritic_game_info.csv')

## Starting with the first videogame data set.
This is the largest, so it will be my basis for merging data.

Just checking out the header.

In [ ]:
vg1.head()

Have to change score to 100 point scale, because all the other are on a 100 point scale. I think it will be nicer to read, rather than moving the 100 point to 10 point.

In [427]:
vg1['critic_score'] = np.where(vg1['critic_score'].isna(), vg1['critic_score'], vg1['critic_score']*10)
vg1['user_score'] = np.where(vg1['user_score'].isna(), vg1['user_score'], vg1['user_score']*10)
vg1.head()

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,3,/games/boxart/full_3351915AmericaFrontccc.jpg,Warriors,Series,Action,Unknown,Omega Force,NaN,NaN,NaN,42.06,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-24
1,4,/games/boxart/full_5741036AmericaFrontccc.jpg,God of War,Series,Action,Sony Computer Entertainment,SIE Santa Monica Studio|Ready at Dawn|Javagrou...,NaN,NaN,NaN,35.05,NaN,NaN,NaN,NaN,NaN,2005-03-22,2020-03-04
2,5,/games/boxart/full_6662824AmericaFrontccc.png,Devil May Cry,Series,Action,Capcom,Capcom|Ninja Theory,NaN,NaN,NaN,22.00,NaN,NaN,NaN,NaN,NaN,2001-10-16,2020-02-03
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,94.0,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
4,7,/games/boxart/full_6800951AmericaFrontccc.jpg,Frogger,Series,Action,Konami,Konami|Parker Brothers|SCE Cambridge Studio|Bl...,NaN,NaN,NaN,20.00,NaN,NaN,NaN,NaN,NaN,1981-10-23,2020-02-26


Checking out the length of dataframe and comparing it to the number of missing review scores. Also removing some columns I am not really interested, specifically sales broken down by territory and "last update." 

In [428]:
print(len(vg1))
print(vg1['critic_score'].isna().sum())
print(vg1['user_score'].isna().sum())
vg1 = vg1.drop(columns=['Unnamed: 0', 'img', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales','last_update'])

vg1.head()

57950
51406
57564


,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,release_date
0,Warriors,Series,Action,Unknown,Omega Force,NaN,NaN,NaN,42.06,NaN,NaN
1,God of War,Series,Action,Sony Computer Entertainment,SIE Santa Monica Studio|Ready at Dawn|Javagrou...,NaN,NaN,NaN,35.05,NaN,2005-03-22
2,Devil May Cry,Series,Action,Capcom,Capcom|Ninja Theory,NaN,NaN,NaN,22.00,NaN,2001-10-16
3,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,94.0,NaN,NaN,20.32,2013-09-17
4,Frogger,Series,Action,Konami,Konami|Parker Brothers|SCE Cambridge Studio|Bl...,NaN,NaN,NaN,20.00,NaN,1981-10-23


Now time to clean the second data set. I have to convert the user scores to float so I can make changes and compare them.

This took a little more work than I thought, but I got it working. I was able to change all of them to float and multiply them by 10.

In [429]:
vg2 = vg2.dropna(subset=['User_Score'])

vg2 = vg2[~vg2.User_Score.str.contains('tbd')]

# vg2.loc['User_Score'] = pd.to_numeric(vg2['User_Score'], errors='ignore')
# vg2['User_Score'] = vg2['User_Score'].astype(float, errors = 'ignore')

vg2["User_Score"] = vg2["User_Score"].map(lambda x: float(x))
vg2['User_Score'] = np.where(vg2['User_Score'].isna(), vg2['User_Score'], (vg2['User_Score']*10))
vg2.info()

#(vg2['User_Score'].unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7590 entries, 0 to 16709
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             7590 non-null   object 
 1   Platform         7590 non-null   object 
 2   Year_of_Release  7463 non-null   float64
 3   Genre            7590 non-null   object 
 4   Publisher        7584 non-null   object 
 5   NA_Sales         7590 non-null   float64
 6   EU_Sales         7590 non-null   float64
 7   JP_Sales         7590 non-null   float64
 8   Other_Sales      7590 non-null   float64
 9   Global_Sales     7590 non-null   float64
 10  Critic_Score     7017 non-null   float64
 11  Critic_Count     7017 non-null   float64
 12  User_Score       7590 non-null   float64
 13  User_Count       7590 non-null   float64
 14  Developer        7579 non-null   object 
 15  Rating           7504 non-null   object 
dtypes: float64(10), object(6)
memory usage: 1008.0+ KB


Here i am doing the majority of cleaning.

In [430]:
print(len(vg2))
print(vg2['Critic_Score'].isna().sum())
print(vg2['User_Score'].isna().sum())

vg2 = vg2.drop(columns=['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Critic_Count','Rating'])
vg2.head()

7590
573
0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7590 entries, 0 to 16709
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             7590 non-null   object 
 1   Platform         7590 non-null   object 
 2   Year_of_Release  7463 non-null   float64
 3   Genre            7590 non-null   object 
 4   Publisher        7584 non-null   object 
 5   NA_Sales         7590 non-null   float64
 6   EU_Sales         7590 non-null   float64
 7   JP_Sales         7590 non-null   float64
 8   Other_Sales      7590 non-null   float64
 9   Global_Sales     7590 non-null   float64
 10  Critic_Score     7017 non-null   float64
 11  Critic_Count     7017 non-null   float64
 12  User_Score       7590 non-null   float64
 13  User_Count       7590 non-null   float64
 14  Developer        7579 non-null   object 
 15  Rating           7504 non-null   object 
dtypes: float64(10), object(6)
memory usage: 1008.0+ 

In [431]:
vg2.info()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,80.0,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,83.0,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,80.0,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,85.0,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,66.0,129.0,Nintendo,E


Dataframe 4 is pretty clean and I am not really going to do any work on this. Will leave it mostly as is, though I might remove some columns later.

In [432]:
print(len(vg4))
print(vg4['metascore'].isna().sum())
print(vg4['user_score'].isna().sum())
vg4.head()

5699
0
0


,game,platform,developer,genre,number_players,rating,release_date,positive_critics,neutral_critics,negative_critics,positive_users,neutral_users,negative_users,metascore,user_score
0,Portal 2,PC,Valve Software,Action,NaN,E10+,"Apr 18, 2011",51,1,0,1700,107,19,95,90
1,The Elder Scrolls V: Skyrim,PC,Bethesda Game Studios,Role-Playing,No Online Multiplayer,M,"Nov 10, 2011",32,0,0,1616,322,451,94,82
2,The Legend of Zelda: Ocarina of Time 3D,3DS,GREZZO,Miscellaneous,No Online Multiplayer,E10+,"Jun 19, 2011",84,1,0,283,20,5,94,90
3,Batman: Arkham City,PC,Rocksteady Studios,Action Adventure,NaN,T,"Nov 21, 2011",27,0,0,240,34,27,91,87
4,Super Mario 3D Land,3DS,Nintendo,Action,No Online Multiplayer,E,"Nov 13, 2011",81,1,0,251,39,11,90,84


Dataframe 5 has the same user score problem, but now that I figured it out, I will do the same thing.

In [433]:
(vg5['Avg_Userscore'].unique())

array(['9.1', '7.4', '7.5', '8.6', '7.9', '9.0', '7.8', '8.3', '6.2',
       '8.5', '8.9', '9.2', '6.4', '7.7', '8.2', '9.3', '8.8', '8.7',
       '6.8', '8.4', '7.0', '8.1', '7.3', '8.0', '5.9', '5.6', '6.5',
       '7.6', '3.4', '5.0', '6.7', '5.8', '6.6', '4.5', '6.3', '7.2',
       '6.9', '5.5', 'tbd', '6.1', '7.1', '5.7', '5.2', '6.0', '4.1',
       'not specified', '3.2', '4.2', '4.9', '3.1', '5.1', '5.4', '4.3',
       '4.4', '4.7', '4.8', '1.6', '5.3', '3.3', '3.5', '3.7', '3.8',
       '3.6', '2.7', '1.7', '2.1', '2.8', '4.6', '4.0', '2.4'],
      dtype=object)

In [434]:
vg5 = vg5.dropna(subset=['Avg_Userscore'])
vg5 = vg5[~vg5.Avg_Userscore.str.contains('tbd|not specified')]
vg5.loc['Avg_Userscore'] = pd.to_numeric(vg5['Avg_Userscore'], errors='coerce')
#print(vg5['Avg_Userscore'].isna().sum())
vg5["Avg_Userscore"] = vg5["Avg_Userscore"].map(lambda x: float(x))
vg5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4914 entries, 0 to Avg_Userscore
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     4913 non-null   float64
 1   Title          4913 non-null   object 
 2   Year           4913 non-null   object 
 3   Publisher      4913 non-null   object 
 4   Genre          4913 non-null   object 
 5   Platform       4913 non-null   object 
 6   Metascore      4913 non-null   object 
 7   Avg_Userscore  4913 non-null   float64
 8   No_Players     4906 non-null   object 
dtypes: float64(2), object(7)
memory usage: 383.9+ KB


In [435]:
print(len(vg5))
print(vg5['Metascore'].isna().sum())
print(vg5['Avg_Userscore'].isna().sum())
vg5 = vg5.drop(columns=['Unnamed: 0', 'Year'])
vg5['Avg_Userscore'] = vg5['Avg_Userscore']*10

vg5.head()

4914
1
1


,Title,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,The Legend of Zelda: Ocarina of Time,Nintendo,Action Adventure;Fantasy,Nintendo64,99,91.0,1 Player
1,Tony Hawk's Pro Skater 2,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,74.0,1-2
2,Grand Theft Auto IV,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,75.0,1 Player
3,SoulCalibur,Namco,Action;Fighting;3D,Dreamcast,98,86.0,1-2
4,Grand Theft Auto IV,RockstarNorth,Action Adventure;Modern;Modern;Open-World,Xbox360,98,79.0,1 Player


Now checking out platform, to clear it up.

In [464]:
np.sort(vg1['console'].unique())
np.sort(vg2['Platform'].unique())
np.sort(vg4['platform'].unique())
# np.sort(vg5['Platform'].unique())

array(['3DS', 'PC', 'PS4', 'PSV', 'Switch', 'WiiU', 'XOne'], dtype=object)

This is making sure all the platform names lines up.

In [438]:
vg4['platform'] = vg4['platform'].replace({'WIIU': 'WiiU', 'XONE': 'XOne', 'VITA': 'PSV'})
vg5['Platform'] = vg5['Platform'].replace({'Dreamcast': 'DC', 'GameBoyAdvance': 'GBA',
                         'GameCube': 'GC', 'Nintendo64': 'N64', 'PlayStation': 'PS',
                         'PlayStation2': 'PS2', 'PlayStation3': 'PS3',
                         'PlayStation4': 'PS4', 'PlayStationVita': 'PSV'
                         , 'Xbox': 'XB', 'Xbox360': 'X360', 'XboxOne': 'XOne'})

Here I will try and merge them. WISH ME LUCK!!

df will be the new dataframe being used.

In [439]:
df = vg1.merge(vg2, left_on=["title", "console"],right_on= ["Name", "Platform"],how='left')

In [440]:
df['critic_score'] = df['critic_score'].combine_first(df['Critic_Score'])

In [441]:
df['user_score'] = df['user_score'].combine_first(df['User_Score'])

In [442]:
df['publisher'] = df['publisher'].combine_first(df['Publisher'])
df['developer'] = df['developer'].combine_first(df['Developer'])

In [443]:
print(df['critic_score'].isna().sum())
print(df['user_score'].isna().sum())
print(len(df))

47705
50339
57953


In [444]:
df = df.drop(columns=['vg_score', 'Name', 'Critic_Score', 'User_Score', 'Developer', 'Publisher', 'Platform'])

In [445]:
df = df.merge(vg4, left_on=["title", "console"],right_on= ["game", "platform"],how='left')

In [446]:
print(df['critic_score'].isna().sum())
print(df['metascore'].isna().sum())
print(df['user_score_x'].isna().sum())
print(df['user_score_y'].isna().sum())
print(df['release_date_y'].isna().sum())
print(df['release_date_x'].isna().sum())

47705
54994
50339
54994
54994
3654


In [447]:
df['critic_score'] = df['critic_score'].combine_first(df['metascore'])

In [448]:
df['user_score'] = df['user_score_x'].combine_first(df['user_score_y'])

In [449]:
print(df['critic_score'].isna().sum())
print(df['user_score'].isna().sum())
print(df['genre_x'].isna().sum())
print(df['Genre'].isna().sum())

45807
48238
0
50541


In [450]:
df['release_date_x'] = df['release_date_x'].combine_first(df['release_date_y'])

In [451]:
df = df.drop(columns=['release_date_y', 'metascore', 'user_score_y','user_score_x', 'developer_y', 'genre_y'])

In [452]:
df = df.drop(columns=['Year_of_Release', 'platform', 'Genre', 'game', 'rating'])
df.head()

,title,console,genre_x,publisher,developer_x,critic_score,total_shipped,total_sales,release_date_x,NA_Sales,...,User_Count,Rating,number_players,positive_critics,neutral_critics,negative_critics,positive_users,neutral_users,negative_users,user_score
0,Warriors,Series,Action,Unknown,Omega Force,NaN,42.06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,God of War,Series,Action,Sony Computer Entertainment,SIE Santa Monica Studio|Ready at Dawn|Javagrou...,NaN,35.05,NaN,2005-03-22,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Devil May Cry,Series,Action,Capcom,Capcom|Ninja Theory,NaN,22.00,NaN,2001-10-16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,94.0,NaN,20.32,2013-09-17,7.02,...,3994.0,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0
4,Frogger,Series,Action,Konami,Konami|Parker Brothers|SCE Cambridge Studio|Bl...,NaN,20.00,NaN,1981-10-23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [453]:
df = df.merge(vg5, left_on=["title", "console"],right_on= ["Title", "Platform"],how='left')

In [454]:
df['user_score'] = df['user_score'].combine_first(df['Avg_Userscore'])

In [455]:
df['critic_score'] = df['critic_score'].combine_first(df['Metascore'])

In [456]:
df['No_Players'] = df['No_Players'].combine_first(df['number_players'])

In [457]:
df['total_sales'] = df['total_sales'].combine_first(df['Global_Sales'])

In [458]:
df = df.drop(columns=['total_shipped'])


In [459]:
#df = df.drop(columns=['Avg_Userscore', 'Metascore', 'Title', 'Publisher',
#                'Platform', 'Genre', 'number_players', 'Global_Sales', 'total_shipped'])
df.head()

,title,console,genre_x,publisher,developer_x,critic_score,total_sales,release_date_x,NA_Sales,EU_Sales,...,neutral_users,negative_users,user_score,Title,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,Warriors,Series,Action,Unknown,Omega Force,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,God of War,Series,Action,Sony Computer Entertainment,SIE Santa Monica Studio|Ready at Dawn|Javagrou...,NaN,NaN,2005-03-22,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Devil May Cry,Series,Action,Capcom,Capcom|Ninja Theory,NaN,NaN,2001-10-16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,94.0,20.32,2013-09-17,7.02,9.09,...,NaN,NaN,82.0,Grand Theft Auto V,RockstarNorth,Modern;Action Adventure;Modern;Open-World,PS3,97,83.0,Up to 16
4,Frogger,Series,Action,Konami,Konami|Parker Brothers|SCE Cambridge Studio|Bl...,NaN,NaN,1981-10-23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [460]:
list(df.columns)

['title',
 'console',
 'genre_x',
 'publisher',
 'developer_x',
 'critic_score',
 'total_sales',
 'release_date_x',
 'NA_Sales',
 'EU_Sales',
 'JP_Sales',
 'Other_Sales',
 'Global_Sales',
 'Critic_Count',
 'User_Count',
 'Rating',
 'number_players',
 'positive_critics',
 'neutral_critics',
 'negative_critics',
 'positive_users',
 'neutral_users',
 'negative_users',
 'user_score',
 'Title',
 'Publisher',
 'Genre',
 'Platform',
 'Metascore',
 'Avg_Userscore',
 'No_Players']

In [461]:
print(len(df))
print(df['critic_score'].isna().sum())
print(df['user_score'].isna().sum())
print(df['total_shipped'].isna().sum())

57953
45458
47626


KeyError: 'total_shipped'

In [462]:
df.to_csv('vg_df.csv', index=False) 

THIS IS ALL WORK SHOPPING STUFF - NOT USED

In [198]:
print(df['critic_score'].isna().sum())
print(df['user_score'].isna().sum())
print(len(df))

46837


KeyError: 'user_score'

In [130]:
print(df['critic_score'].isna().sum())
print(df['user_score'].isna().sum())

KeyError: 'critic_score'

In [10]:
# Just checking out how many unique publishers there are.
len(pd.unique(vg_chartz['publisher']))

3261

In [44]:
vg3.head()

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,3,/games/boxart/full_3351915AmericaFrontccc.jpg,Warriors,Series,Action,Unknown,Omega Force,NaN,NaN,NaN,42.06,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-24
1,4,/games/boxart/full_5741036AmericaFrontccc.jpg,God of War,Series,Action,Sony Computer Entertainment,SIE Santa Monica Studio|Ready at Dawn|Javagrou...,NaN,NaN,NaN,35.05,NaN,NaN,NaN,NaN,NaN,2005-03-22,2020-03-04
2,5,/games/boxart/full_6662824AmericaFrontccc.png,Devil May Cry,Series,Action,Capcom,Capcom|Ninja Theory,NaN,NaN,NaN,22.00,NaN,NaN,NaN,NaN,NaN,2001-10-16,2020-02-03
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
4,7,/games/boxart/full_6800951AmericaFrontccc.jpg,Frogger,Series,Action,Konami,Konami|Parker Brothers|SCE Cambridge Studio|Bl...,NaN,NaN,NaN,20.00,NaN,NaN,NaN,NaN,NaN,1981-10-23,2020-02-26


In [38]:
print(len(vg2))
print(vg2['Global_Sales'].isna().sum())
print(vg2['Critic_Score'].isna().sum())
print(vg2['User_Score'].isna().sum())

16719
0
8582
6704


https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas

Based on the number of publishers, I will need to find a way to link them to the company that owns them.

In [17]:
# removing series and visual novels, might use merge series later as a data point, so making its own df
vg_copy = vg_copy[vg_copy.console != 'Series']
vg_copy = vg_copy[vg_copy.genre != 'Visual Novel']
vg_series = vg_chartz[vg_chartz.console == 'Series']

In [19]:
vg_series.head

<bound method NDFrame.head of        Unnamed: 0                                            img  \
0               3  /games/boxart/full_3351915AmericaFrontccc.jpg   
1               4  /games/boxart/full_5741036AmericaFrontccc.jpg   
2               5  /games/boxart/full_6662824AmericaFrontccc.png   
4               7  /games/boxart/full_6800951AmericaFrontccc.jpg   
16             19  /games/boxart/full_8886048AmericaFrontccc.jpg   
...           ...                                            ...   
54132       54978  /games/boxart/full_7404545AmericaFrontccc.jpg   
54133       54979  /games/boxart/full_7398718AmericaFrontccc.png   
54139       54985  /games/boxart/full_5242617AmericaFrontccc.jpg   
54142       54988  /games/boxart/full_9421541AmericaFrontccc.jpg   
54189       55035  /games/boxart/full_9984096AmericaFrontccc.jpg   

                               title console     genre  \
0                           Warriors  Series    Action   
1                         God of War 

280 series, which should be workable. I probably wont work with all of them, since some of them probably are just the same game released for different systems (eg. Tetris, Frogger, The Binding of Isaac, etc.) For these games, the total shipped could be an interesting stat to consider.

In [14]:
# checking out the header
vg_copy.head

<bound method NDFrame.head of        Unnamed: 0                                             img  \
3               6   /games/boxart/full_6510540AmericaFrontccc.jpg   
5               8   /games/boxart/full_5563178AmericaFrontccc.jpg   
6               9                    /games/boxart/3887656ccc.jpg   
7              10   /games/boxart/full_2804791AmericaFrontccc.jpg   
8              11                     /games/boxart/827563ccc.jpg   
...           ...                                             ...   
57523       58420   /games/boxart/full_6832495AmericaFrontccc.jpg   
57524       58421  /games/boxart/full_8405567AmericaFrontccc.jpeg   
57525       58422   /games/boxart/full_1661793AmericaFrontccc.png   
57526       58423   /games/boxart/full_5465265AmericaFrontccc.jpg   
57527       58424   /games/boxart/full_6380526AmericaFrontccc.jpg   

                                                 title console     genre  \
3                                   Grand Theft Auto V     PS3   

With 57428 rows and 18 columns, there is definitely something to look at. Will be focusing on critic score and total sales (or shipped) for continuous variables.

Categorical will be publisher, console, and genre. Might add some series analysis later.

Will look at stocks for a few of the publishers, probably focusing on the ones that only produce game software. (No Nintendo, Sony, or Microsoft.) Will probably look at Activision, EA, Ubisoft, Take-Two, and Square-Enix. The problem with that will probably be combining some of the companies that have multiple publishers (eg. Take-Two)

In [15]:
# counting publishers
vg_copy['publisher'].nunique()

3240

Count of unique publishers = 3240: might need to delve into that more later, removing publishers that focus on one or two titles. It could be an interesting data point later, seeing if a publisher correlates with another measurement variable.

In [56]:
# grouping stuff

group = vg_copy.groupby(['genre'])
group.agg(['mean', 'std', 'count'])['critic_score']

,mean,std,count
genre,,,
Action,6.840234,1.575293,1024
Action-Adventure,7.665657,1.239170,99
Adventure,7.329188,1.455842,394
Board Game,5.300000,NaN,1
Education,7.500000,NaN,1
Fighting,7.388415,1.383228,328
MMO,7.810000,1.408269,10
Misc,6.885799,1.517741,338
Music,7.973077,0.961065,26


In [57]:
group.agg(['mean', 'std', 'count'])['total_sales']

,mean,std,count
genre,,,
Action,0.409260,1.048028,2880
Action-Adventure,0.556816,1.202643,267
Adventure,0.177714,0.396895,1920
Board Game,0.110000,0.164621,3
Education,0.090000,NaN,1
Fighting,0.400277,0.610933,903
MMO,0.370625,0.520452,32
Misc,0.289230,0.588833,2039
Music,0.352752,0.708438,149


In [100]:
# This was first try, using merging. Moved to using isin, below.
# group2 = vg_copy.groupby(['console'])
# test = group2.agg(['count'])['critic_score']
# blah = test > 100
# blah.reset_index()
# new_df = pd.merge(vg_copy, blah,  how='left', left_on=['console'], right_on=['console'])

In [122]:
# grouping it by column that interested in (console)
group2 = vg_copy.groupby(['console'])

# counting times that column (console here) (for each of its unique values) have a given value (critic score)
test = group2.agg(['count'])['critic_score']

# choosing the number of critic scores needed to be included
blah = test > 100

#only keeping the ones that have over that number of columns.
boo = blah[blah['count'] == True]

# now checking if those columns (consoles) are in the new 
vg_copy[vg_copy['console'].isin(boo.index)]

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
5,8,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,NaN,9.7,NaN,NaN,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
6,9,/games/boxart/3887656ccc.jpg,Grand Theft Auto: San Andreas,PS2,Action,Rockstar Games,Rockstar North,NaN,9.5,NaN,17.30,NaN,NaN,NaN,NaN,NaN,2004-10-26,NaN
7,10,/games/boxart/full_2804791AmericaFrontccc.jpg,Uncharted 4: A Thief's End,PS4,Action,Sony Interactive Entertainment,Naughty Dog,NaN,9.2,NaN,16.25,NaN,NaN,NaN,NaN,NaN,2016-05-10,2018-09-23
8,11,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,NaN,9.6,NaN,NaN,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57521,58415,/games/boxart/full_3900223AmericaFrontccc.jpg,XCOM 2 Collection,NS,Strategy,2K Games,Firaxis Games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-29,2020-03-31
57522,58416,/games/boxart/full_5852173AmericaFrontccc.jpg,XCOM: Chimera Squad,PC,Strategy,2K Games,Firaxis Games,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-24,2020-04-15
57524,58421,/games/boxart/full_8405567AmericaFrontccc.jpeg,Yu Yu Hakusho Tournament Tactics,GBA,Strategy,Atari,Sensory Sweep Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-11-18,2017-12-30
57525,58422,/games/boxart/full_1661793AmericaFrontccc.png,Yu-Gi-Oh! Legacy of the Duelist,PS4,Strategy,Konami,Other Ocean Interactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-31,2019-04-20


In [123]:
test

,count
console,
2600,0
3DO,0
3DS,220
5200,0
7800,0
...,...
XOne,58
XSX,0
ZXS,0


Created a function here that removes rows if not enough specific features.

In [20]:
def enough_stuff(dat_fr, uniq_col, count_col, cnt_val):
    """
    DOC STRING
    This function takes a data frame, a column of categoricals
    to count, and the number of items from that column that are 
    in rows needed to be kept. This is basically just removing 
    items that dont show up enough to study.
    
    Args:
    dat_fr: The data frame that will be looked at.
    uniq_col: The column of the data frame that the function will look at
    count_col: The column with the counting objects.
    cnt_val: The number of items needed to be included.
    
    
    Return:
    A new data frame with just the rows that have 
    those requested values in the column.
    A print out of how many rows were removed.
    """
    start_len = len(dat_fr)
    func_group = dat_fr.groupby([uniq_col])[count_col]
    func_TorF = func_group.agg(['count']) > cnt_val
    func_JustT = func_TorF[func_TorF['count'] == True]
    final_df = dat_fr[dat_fr[uniq_col].isin(func_JustT.index)]
    print(f'{start_len - len(final_df)} of {start_len} have been removed. You now have {len(final_df)}')
    return (final_df)
    

In [21]:
vg_copy = enough_stuff(vg_copy, 'console', 'critic_score', 100)

16032 of 57248 have been removed. You now have 41216


In [22]:
vg_copy = enough_stuff(vg_copy, 'genre', 'critic_score', 90)

1011 of 41216 have been removed. You now have 40205


In [41]:
#vg_copy[vg_copy.critic_score != 'NaN']
print(len(vg_copy))
print(vg_copy['critic_score'].isna().sum())
print(vg_copy['user_score'].isna().sum())
print(vg_copy['total_sales'].isna().sum())

40205
34264
39859
23422


In [51]:
vg_copy[vg_copy['title'].str.contains('Grand Theft Auto')]

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
5,8,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,NaN,9.7,NaN,NaN,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
6,9,/games/boxart/3887656ccc.jpg,Grand Theft Auto: San Andreas,PS2,Action,Rockstar Games,Rockstar North,NaN,9.5,NaN,17.3,NaN,NaN,NaN,NaN,NaN,2004-10-26,NaN
8,11,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,NaN,9.6,NaN,NaN,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
9,12,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,NaN,NaN,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21156,21486,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,And,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,NaN
21157,21487,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,OSX,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-09-01,NaN
21158,21488,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,WinP,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,NaN
21159,21489,/games/boxart/default.jpg,Grand Theft Auto: The Trilogy,PSV,Misc,Kadokawa Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,NaN


In [101]:

new_df = new_df[new_df['count'] == True]
new_df

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update,count
0,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN,True
1,8,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,NaN,9.7,NaN,NaN,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03,True
2,9,/games/boxart/3887656ccc.jpg,Grand Theft Auto: San Andreas,PS2,Action,Rockstar Games,Rockstar North,NaN,9.5,NaN,17.30,NaN,NaN,NaN,NaN,NaN,2004-10-26,NaN,True
3,10,/games/boxart/full_2804791AmericaFrontccc.jpg,Uncharted 4: A Thief's End,PS4,Action,Sony Interactive Entertainment,Naughty Dog,NaN,9.2,NaN,16.25,NaN,NaN,NaN,NaN,NaN,2016-05-10,2018-09-23,True
4,11,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,NaN,9.6,NaN,NaN,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57241,58415,/games/boxart/full_3900223AmericaFrontccc.jpg,XCOM 2 Collection,NS,Strategy,2K Games,Firaxis Games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-29,2020-03-31,True
57242,58416,/games/boxart/full_5852173AmericaFrontccc.jpg,XCOM: Chimera Squad,PC,Strategy,2K Games,Firaxis Games,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-24,2020-04-15,True
57244,58421,/games/boxart/full_8405567AmericaFrontccc.jpeg,Yu Yu Hakusho Tournament Tactics,GBA,Strategy,Atari,Sensory Sweep Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-11-18,2017-12-30,True
57245,58422,/games/boxart/full_1661793AmericaFrontccc.png,Yu-Gi-Oh! Legacy of the Duelist,PS4,Strategy,Konami,Other Ocean Interactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-31,2019-04-20,True


In [68]:
counts = vg_copy['console'].nunique()
counts
# filtered = counts[counts > 100]
# vg3 = vg_copy[vg_copy['console'].isin(filtered.index)]
# group2 = vg3.groupby(['console'])
# group2.agg(['mean', 'std', 'count'])['critic_score']

77

In [72]:
vg_copy=vg_copy.loc[:,vg_copy.nunique()>100]
vg_copy

""


In [74]:
vg_copy = vg_copy.groupby('console')[].nunique()

KeyError: 'console'

In [99]:
# creating a groupby for publisher, so see stuff
vg_copy.groupby('publisher')

In [51]:
vg_copy[vg_copy['title'].str.contains('Grand Theft Auto')]


# df = pd.concat([vg1, vg3]).drop_duplicates(subset=['title', 'console']).reset_index(drop=True)

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
5,8,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,NaN,9.7,NaN,NaN,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
6,9,/games/boxart/3887656ccc.jpg,Grand Theft Auto: San Andreas,PS2,Action,Rockstar Games,Rockstar North,NaN,9.5,NaN,17.3,NaN,NaN,NaN,NaN,NaN,2004-10-26,NaN
8,11,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,NaN,9.6,NaN,NaN,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
9,12,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,NaN,NaN,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21156,21486,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,And,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,NaN
21157,21487,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,OSX,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-09-01,NaN
21158,21488,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,WinP,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,NaN
21159,21489,/games/boxart/default.jpg,Grand Theft Auto: The Trilogy,PSV,Misc,Kadokawa Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,NaN


In [129]:
vg_df = vg_df[~vg_df.user_score.str.contains('tbd')]
# vg_df["user_score"] = vg_df["user_score"].map(lambda x: float(x))

AttributeError: Can only use .str accessor with string values!